<a href="https://colab.research.google.com/github/SahanaSwamy/Music-Genre-Recognition/blob/master/Music_Genre_Recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**MUSIC GENRE RECOGNITION**

##**IMPORTING LIBRARIES**

In [ ]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler


import keras

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!ls

sample_data


In [ ]:
##mouting the google drive

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [ ]:
##this is the current directory

%cd /gdrive/My\ Drive/genres

/gdrive/My Drive/genres


In [ ]:
!ls

 anysong.ipynb	  hiphop	 newdata.csv	    scaler.pickle
 blues		  img_data	'newsong (1).mp3'   X1.pickle
 classical	  jazz		 newsong.mp3	    X2.pickle
 classical.mp3	  metal		 pop		    X.pickle
 country	  my_model.h5	 reggae		    y1.pickle
 data.csv	  newdata1.csv	 rock		    y2.pickle
 disco		  newdata2.csv	 rough1.ipynb
 FinalMus.model   newdata3.csv	 scaler


##**SPECTOGRAM CREATION**

In [ ]:
##Creating the spectogram for the songs

cmap = plt.get_cmap('inferno')
import time
plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres: 
    for filename in os.listdir(g):
      print(filename)
      songname = f'{g}/{filename}'
      print(songname)
      print('filename is')
      print(filename)
      y, sr = librosa.load(songname, mono=True, duration=5)
      plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
      plt.axis('off');
      time.sleep(1)
      plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
      plt.clf()

In [ ]:
##creating the header for the csv file

header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

##**CSV FILE CREATION**

In [ ]:
##Creating the csv file from the spectograms

file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(g):
        songname = f'{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [ ]:
data=pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.au,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,42.351029,-6.376457,18.618875,-13.697911,15.344630,-12.285266,10.980491,-8.324323,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,blues.00001.au,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,35.874684,2.916038,21.523725,-8.554703,23.358672,-10.103616,11.903744,-5.560387,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,blues.00002.au,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,31.689014,-13.987036,25.754761,-13.649586,11.629271,-11.780589,9.706442,-13.123111,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00003.au,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,26.871927,1.754463,14.238344,-4.830882,9.297965,-0.757741,8.149012,-3.196314,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.au,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,22.153301,-32.489270,10.864512,-23.357929,0.503117,-11.805831,1.206804,-13.083820,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues


In [ ]:
data.shape

(1000, 28)

In [ ]:
data = data.drop(['filename'],axis=1)

In [ ]:
##creating y variable as an output 

genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [ ]:
y.shape

(1000,)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [ ]:
##pickling the scaler object to be used in anysong.ipynb

import pickle
with open(r"scaler.pickle", "wb") as output_file:
  pickle.dump(scaler, output_file)

In [ ]:
X.shape

(1000, 26)

In [ ]:
X

array([[-0.35174834, -0.01072298, -0.58330334, ..., -0.23719158,
         0.00761145,  0.60349813],
       [-0.46146547, -0.53326615, -0.93906628, ..., -0.05518978,
         0.5438236 ,  0.42403528],
       [-0.18448375,  0.68001209, -0.90741936, ..., -0.60070707,
        -0.29428464, -0.29511278],
       ...,
       [ 0.65431764, -0.75110651, -0.17418012, ...,  0.76028053,
        -2.73474414, -0.26387449],
       [-0.19983702, -0.71651358, -1.12235633, ...,  0.2717664 ,
        -0.72311185, -0.64936228],
       [-0.2507021 , -1.16473892, -0.82782084, ..., -0.12506872,
         0.08171799,  0.58748963]])

In [ ]:
##creating a test and train split 80-20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train[10]

array([-0.25268682,  0.20586734, -0.45149864, -0.07150224, -0.18076424,
       -0.76307272,  0.13403738,  0.36388879,  0.31485445,  0.33047797,
       -0.16628522,  1.6924497 , -1.25355844,  0.93180651, -0.25745713,
        0.90205058, -0.11848772,  0.70364263, -0.35118038,  0.90488229,
       -1.18155211, -0.14040636, -1.93635414, -1.3994824 , -0.05014615,
       -0.52935688])

##**SEQUENTIAL MODEL**

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

W0814 07:29:01.894109 140281916925824 module_wrapper.py:137] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

W0814 07:29:05.062322 140281916925824 module_wrapper.py:137] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [ ]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

W0814 07:29:07.026734 140281916925824 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1423: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
800/800 [==============================] - 0s 342us/step - loss: 2.2221 - acc: 0.1975
Epoch 2/20
800/800 [==============================] - 0s 43us/step - loss: 1.8968 - acc: 0.3550
Epoch 3/20
800/800 [==============================] - 0s 40us/step - loss: 1.6461 - acc: 0.3962
Epoch 4/20
800/800 [==============================] - 0s 40us/step - loss: 1.4524 - acc: 0.4637
Epoch 5/20
800/800 [==============================] - 0s 38us/step - loss: 1.3027 - acc: 0.5475
Epoch 6/20
800/800 [==============================] - 0s 39us/step - loss: 1.2074 - acc: 0.5813
Epoch 7/20
800/800 [==============================] - 0s 42us/step - loss: 1.1132 - acc: 0.6313
Epoch 8/20
800/800 [==============================] - 0s 38us/step - loss: 1.0359 - acc: 0.6638
Epoch 9/20
800/800 [==============================] - 0s 43us/step - loss: 0.9644 - acc: 0.7025
Epoch 10/20
800/800 [==============================] - 0s 45us/step - loss: 0.9209 - acc: 0.6938
Epoch 11/20
800/800 [=================

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 239us/step


In [ ]:
print('test_acc: ',test_acc)

test_acc:  0.635


In [ ]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [ ]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results, final_accuracy = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 0s 699us/step - loss: 2.3192 - acc: 0.1017 - val_loss: 2.2028 - val_acc: 0.1750
Epoch 2/30
600/600 [==============================] - 0s 69us/step - loss: 2.1527 - acc: 0.2617 - val_loss: 2.0898 - val_acc: 0.2450
Epoch 3/30
600/600 [==============================] - 0s 68us/step - loss: 2.0100 - acc: 0.3667 - val_loss: 1.9825 - val_acc: 0.2550
Epoch 4/30
600/600 [==============================] - 0s 65us/step - loss: 1.8642 - acc: 0.3883 - val_loss: 1.8845 - val_acc: 0.2750
Epoch 5/30
600/600 [==============================] - 0s 70us/step - loss: 1.7261 - acc: 0.4033 - val_loss: 1.7885 - val_acc: 0.3500
Epoch 6/30
600/600 [==============================] - 0s 67us/step - loss: 1.5781 - acc: 0.4567 - val_loss: 1.7084 - val_acc: 0.3600
Epoch 7/30
600/600 [==============================] - 0s 66us/step - loss: 1.4463 - acc: 0.5283 - val_loss: 1.6690 - val_acc: 0.3850
Epoch 8/30
600/600 [==

In [ ]:
results

1.1344644927978516

In [ ]:
final_accuracy

0.62

In [ ]:

predictions = model.predict_proba(X_test)

In [ ]:
##testing results
predictions

array([[2.2052022e-02, 1.9365858e-05, 4.8412569e-03, ..., 9.5206611e-05,
        1.3424294e-02, 1.3988456e-02],
       [7.9929050e-05, 9.8646331e-01, 1.8581332e-03, ..., 1.2649713e-03,
        4.6634213e-03, 1.0940355e-03],
       [6.7730318e-05, 6.5421823e-06, 2.4778975e-04, ..., 1.2087699e-03,
        1.9879304e-03, 7.3835459e-03],
       ...,
       [5.8634585e-04, 3.4400829e-04, 7.5144127e-02, ..., 6.0119599e-02,
        1.4644089e-01, 7.3174648e-02],
       [5.6374422e-04, 6.7272787e-03, 1.4532682e-02, ..., 5.4786773e-04,
        4.3406291e-03, 9.0519585e-02],
       [2.6209694e-03, 2.4114423e-05, 2.8983572e-01, ..., 3.8025039e-05,
        1.0085267e-02, 6.8461990e-01]], dtype=float32)

In [ ]:
predictions.shape

(200, 10)

In [ ]:
!pip install h5py pyyaml
!pip install tf_nightly

In [ ]:

model.save('my_model.h5')